In [47]:
# lesson_functions

import numpy as np
import cv2
from skimage.feature import hog
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
import glob
from skimage.feature import hog

def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)

def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, size=(32, 32)):
    #TODO: Debug this
    print(img.shape)
    print(img[:,:,0].shape)
    color1 = cv2.resize(img[:,:,0], size).ravel()
    color2 = cv2.resize(img[:,:,1], size).ravel()
    color3 = cv2.resize(img[:,:,2], size).ravel()
    return np.hstack((color1, color2, color3))
                        
# Define a function to compute color histogram features 
# TODO: NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features


#Feature extraction

def extract_features(imgs, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, hist_range=(0, 256)):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
        
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
        else: feature_image = np.copy(image)      
            
        # Apply bin_spatial() to get spatial color features
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        # Apply color_hist() also with a color space option now
        hist_features = color_hist(feature_image, nbins=hist_bins, bins_range=hist_range)
        
        #transform color space
        image_transformed = convert_color(feature_image, conv='RGB2YCrCb')
        ch1 = image_transformed[:,:,0]
        ch2 = image_transformed[:,:,1]
        ch3 = image_transformed[:,:,2]
        # Compute individual channel HOG features for the entire image
        hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False).ravel()
        hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False).ravel()
        hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False).ravel()
        hog_features = np.hstack((hog1, hog2, hog3))
        
        all_features = np.hstack((spatial_features, hist_features, hog_features))
                
        # Fit a per-column scaler
        X_scaler = StandardScaler().fit(all_features.reshape(1, -1))
        # Apply the scaler to X
        scaled_features = X_scaler.transform(all_features.reshape(1, -1))
        
        # Append the new feature vector to the features list
        features.append(scaled_features)
    # Return list of feature vectors
    return features

In [48]:
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

# Read in car and non-car images
images = glob.iglob('./data/subset_data/**/*.jpeg', recursive=True)
cars = []
notcars = []
for image in images:
    if 'image' in image or 'extra' in image:
        notcars.append(image)
    else:
        cars.append(image)
        
# Define the labels vector
y = np.hstack((np.ones(len(cars)), np.zeros(len(notcars))))

### TODO: Tweak these parameters and see how the results change.
color_space = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = 0 # Can be 0, 1, 2, or "ALL"
spatial = (16, 16) # Spatial binning dimensions
histbin = 16    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off
y_start_stop = [None, None] # Min and max in y to search in slide_window()
# Get features
car_features = extract_features(cars, cspace=color_space, spatial_size=spatial,
                        hist_bins=histbin, hist_range=(0, 256))
notcar_features = extract_features(notcars, cspace=color_space, spatial_size=(spatial, spatial),
                        hist_bins=histbin, hist_range=(0, 256))

rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

svc = LinearSVC()
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()

print(round(t2-t, 2), 'Seconds to train SVC...')
# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)



(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64, 64, 3)
(64, 64)
(64

TypeError: an integer is required (got type tuple)